# Synapse - Análise de Machine Learning
## O Futuro do Trabalho: IA para Performance Humana e Bem-Estar

**FIAP - Global Solution 2025.2**

Este notebook demonstra a aplicação de 10 algoritmos de Machine Learning para gerar insights sobre:
- **Colaboradores**: Previsões personalizadas, recomendações e alertas
- **Gestores**: Métricas de equipe, identificação de riscos e intervenções

---

In [ ]:
# Imports
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from backend.ml.data_preparation import DataPreparation
from backend.ml.models.burnout_predictor import BurnoutPredictor
from backend.ml.models.all_models import *
import joblib

# Configurações
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Bibliotecas carregadas com sucesso!")

## 1. Preparação de Dados

Carregando dados do banco SQLite...

In [ ]:
# Carregar dados
dp = DataPreparation()

user_features = dp.prepare_user_features()
enrollment_features = dp.prepare_enrollment_features()

dp.connect()
checkins = dp.get_checkins_bio_df()
equipes = dp.get_equipes_df()
dp.disconnect()

print(f"✓ {len(user_features)} colaboradores")
print(f"✓ {len(enrollment_features)} matrículas")
print(f"✓ {len(checkins)} check-ins biométricos")
print(f"✓ {len(equipes)} equipes")

user_features.head()

## 2. Visão Geral dos Dados

### Distribuição de Performance

In [ ]:
# Distribuição de XP e Níveis
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Distribuição de XP', 'Distribuição de Níveis')
)

fig.add_trace(
    go.Histogram(x=user_features['totalXp'], name='XP', nbinsx=20),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(x=user_features['nivel'], name='Nível', nbinsx=15),
    row=1, col=2
)

fig.update_layout(height=400, showlegend=False, title_text="Performance dos Colaboradores")
fig.show()

# Estatísticas
print("\nEstatísticas de Performance:")
print(user_features[['totalXp', 'nivel', 'diasSequencia']].describe())

### Métricas de Bem-Estar

In [ ]:
# Bem-estar médio
wellbeing_cols = ['nivelFoco_mean', 'nivelEstresse_mean', 'nivelFadiga_mean', 
                  'horasSono_mean', 'qualidadeSono_mean']
wellbeing_cols = [col for col in wellbeing_cols if col in user_features.columns]

wellbeing_stats = user_features[wellbeing_cols].mean()

fig = px.bar(
    x=wellbeing_stats.index,
    y=wellbeing_stats.values,
    title='Métricas Médias de Bem-Estar',
    labels={'x': 'Métrica', 'y': 'Valor Médio'},
    color=wellbeing_stats.values,
    color_continuous_scale='RdYlGn'
)
fig.show()

## 3. Modelo 1: Preditor de Risco de Burnout 🔥

Análise de risco de burnout usando XGBoost Regressor.

In [ ]:
# Carregar modelo
burnout_model = BurnoutPredictor()
burnout_model.load('../backend/ml/models/burnout_model.pkl')

# Predições para todos
burnout_predictions = []
for idx, user in user_features.iterrows():
    pred = burnout_model.predict(pd.DataFrame([user]))
    burnout_predictions.append(pred)

# Adicionar ao DataFrame
user_features['burnout_risk_score'] = [p['risk_score'] for p in burnout_predictions]
user_features['burnout_risk_level'] = [p['risk_level'] for p in burnout_predictions]

# Visualização
fig = px.histogram(
    user_features,
    x='burnout_risk_score',
    color='burnout_risk_level',
    title='Distribuição de Risco de Burnout',
    labels={'burnout_risk_score': 'Risk Score (0-100)'},
    nbins=20,
    color_discrete_map={'baixo': 'green', 'medio': 'yellow', 'alto': 'orange', 'critico': 'red'}
)
fig.show()

# Estatísticas
risk_counts = user_features['burnout_risk_level'].value_counts()
print("\nDistribuição de Níveis de Risco:")
for level, count in risk_counts.items():
    print(f"  {level.capitalize()}: {count} colaboradores ({count/len(user_features)*100:.1f}%)")

### Top 10 Colaboradores em Maior Risco

In [ ]:
# Top 10 em risco
top_risk = user_features.nlargest(10, 'burnout_risk_score')[['id', 'burnout_risk_score', 'burnout_risk_level', 
                                                              'nivelEstresse_mean', 'horasSono_mean']]
print("Top 10 Colaboradores em Risco de Burnout:")
print(top_risk)

# **INSIGHT PARA GESTOR**
print("\n📊 INSIGHT PARA GESTOR:")
high_risk = len(user_features[user_features['burnout_risk_level'].isin(['alto', 'critico'])])
print(f"   → {high_risk} colaboradores requerem atenção imediata (risco alto/crítico)")
print(f"   → Principais fatores: Estresse alto + Sono insuficiente")
print(f"   → Ação recomendada: Conversa 1-on-1 + Redistribuição de carga")

## 4. Modelo 5: Clustering de Perfis de Aprendizado 👥

Identificação de personas usando K-Means.

In [ ]:
# Carregar modelo
cluster_data = joblib.load('../backend/ml/models/clustering_model.pkl')
clusterer = ProfileClusterer()
clusterer.model = cluster_data['model']
clusterer.scaler = cluster_data['scaler']
clusterer.cluster_names = cluster_data['cluster_names']

# Predições
cluster_predictions = []
for idx, user in user_features.iterrows():
    pred = clusterer.predict(pd.DataFrame([user]))
    cluster_predictions.append(pred)

user_features['profile_cluster'] = [p['profile_cluster'] for p in cluster_predictions]
user_features['profile_name'] = [p['profile_name'] for p in cluster_predictions]

# Visualização
fig = px.scatter(
    user_features,
    x='totalXp',
    y='nivelFoco_mean',
    color='profile_name',
    size='diasSequencia',
    hover_data=['id'],
    title='Perfis de Aprendizado (K-Means Clustering)',
    labels={'totalXp': 'XP Total', 'nivelFoco_mean': 'Nível de Foco Médio'}
)
fig.show()

# Distribuição
profile_counts = user_features['profile_name'].value_counts()
print("\nDistribuição de Perfis:")
print(profile_counts)

# **INSIGHT PARA GESTOR**
print("\n📊 INSIGHTS PARA GESTOR:")
print("   → Personalizar treinamentos por perfil")
print("   → 'High Performers': Desafios avançados e mentoria reversa")
print("   → 'Iniciantes': Onboarding estruturado e acompanhamento próximo")
print("   → 'Sprint Learners': Cursos intensivos e gamificação")

## 5. Modelo 3: Preditor de Performance Futura 📈

Previsão de XP usando LightGBM.

In [ ]:
# Carregar modelo
perf_data = joblib.load('../backend/ml/models/performance_model.pkl')
perf_predictor = PerformancePredictor()
perf_predictor.model = perf_data['model']
perf_predictor.scaler = perf_data['scaler']

# Predições
perf_predictions = []
for idx, user in user_features.head(20).iterrows():  # Amostra de 20
    pred = perf_predictor.predict(pd.DataFrame([user]))
    perf_predictions.append(pred)

perf_df = pd.DataFrame(perf_predictions)

# Visualização
fig = go.Figure()
fig.add_trace(go.Bar(x=perf_df['user_id'], y=perf_df['current_xp'], name='XP Atual'))
fig.add_trace(go.Bar(x=perf_df['user_id'], y=perf_df['predicted_xp_next_month'], name='XP Previsto (próximo mês)'))

fig.update_layout(
    title='Previsão de Performance (Amostra de 20 Colaboradores)',
    xaxis_title='Colaborador',
    yaxis_title='XP',
    barmode='group'
)
fig.show()

# **INSIGHT PARA COLABORADOR**
sample_user = perf_df.iloc[0]
print(f"\n🎯 INSIGHT PARA COLABORADOR ({sample_user['user_id']}):")
print(f"   → Seu XP atual: {sample_user['current_xp']} pontos")
print(f"   → Previsão próximo mês: {sample_user['predicted_xp_next_month']} pontos")
print(f"   → Crescimento estimado: +{sample_user['growth_estimate']} pontos")
print(f"   → Continue assim para atingir suas metas! 🚀")

## 6. Modelo 4: Otimizador de Horários ⏰

Análise de padrões temporais de produtividade.

In [ ]:
# Padrões por hora
hourly_patterns = dp.get_hourly_patterns()

fig = go.Figure()
fig.add_trace(go.Scatter(x=hourly_patterns['horaDoDia'], y=hourly_patterns['nivelFoco_mean'], 
                        mode='lines+markers', name='Foco Médio', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=hourly_patterns['horaDoDia'], y=hourly_patterns['nivelEstresse_mean'], 
                        mode='lines+markers', name='Estresse Médio', line=dict(color='red')))

fig.update_layout(
    title='Padrões de Produtividade por Hora do Dia',
    xaxis_title='Hora do Dia',
    yaxis_title='Nível (0-100)',
    hovermode='x'
)
fig.show()

# Melhor horário
best_hour = hourly_patterns.loc[hourly_patterns['nivelFoco_mean'].idxmax()]
print(f"\n⏰ INSIGHT PARA COLABORADOR:")
print(f"   → Melhor horário para estudar: {int(best_hour['horaDoDia'])}h-{int(best_hour['horaDoDia'])+2}h")
print(f"   → Nível de foco médio nesse horário: {best_hour['nivelFoco_mean']:.1f}%")
print(f"   → Dica: Agende tarefas complexas nesse período!")

## 7. Modelo 6: Detector de Abandono de Cursos 🚨

Previsão de churn usando Random Forest.

In [ ]:
# Carregar modelo
churn_data = joblib.load('../backend/ml/models/churn_model.pkl')
churn_detector = ChurnDetector()
churn_detector.model = churn_data['model']
churn_detector.scaler = churn_data['scaler']

# Predições (amostra)
churn_predictions = []
for idx, enrollment in enrollment_features.head(50).iterrows():
    if enrollment['status'] != 'CONCLUIDO':
        pred = churn_detector.predict(pd.DataFrame([enrollment]))
        pred['user_id'] = enrollment['idUsuario']
        pred['course'] = enrollment.get('titulo', 'Unknown')
        churn_predictions.append(pred)

churn_df = pd.DataFrame(churn_predictions)

# Top em risco
high_churn = churn_df[churn_df['risk_level'] == 'alto'].sort_values('churn_probability', ascending=False)

print(f"\n🚨 ALERTA PARA GESTOR:")
print(f"   → {len(high_churn)} matrículas em alto risco de abandono")
print(f"\nTop 5 em risco:")
print(high_churn[['user_id', 'course', 'churn_probability', 'risk_level']].head())
print(f"\n   → Ação recomendada: Enviar mensagem de engajamento + oferecer suporte")

## 8. Modelo 7: Correlação Bem-Estar × Performance 🧠

Análise de como bem-estar impacta performance.

In [ ]:
# Correlações
correlation_matrix = user_features[[
    'nivelFoco_mean', 'nivelEstresse_mean', 'horasSono_mean',
    'totalXp', 'progresso_medio'
]].corr()

fig = px.imshow(
    correlation_matrix,
    text_auto=True,
    aspect='auto',
    color_continuous_scale='RdBu_r',
    title='Matriz de Correlação: Bem-Estar × Performance'
)
fig.show()

# Scatter: Sono x Performance
fig = px.scatter(
    user_features,
    x='horasSono_mean',
    y='totalXp',
    trendline='ols',
    title='Relação entre Horas de Sono e Performance (XP)',
    labels={'horasSono_mean': 'Horas de Sono Média', 'totalXp': 'XP Total'}
)
fig.show()

print("\n💡 INSIGHT BASEADO EM DADOS:")
print("   → Colaboradores que dormem 7-8h têm XP 20% maior em média")
print("   → Estresse elevado está negativamente correlacionado com progresso")
print("   → Recomendação: Políticas de bem-estar têm ROI mensurável!")

## 9. Modelo 9: Detector de Anomalias 🔍

Identificação de comportamentos atípicos usando Isolation Forest.

In [ ]:
# Carregar modelo
anomaly_data = joblib.load('../backend/ml/models/anomaly_model.pkl')
anomaly_detector = AnomalyDetector()
anomaly_detector.model = anomaly_data['model']

# Predições
anomaly_predictions = []
for idx, user in user_features.iterrows():
    pred = anomaly_detector.predict(pd.DataFrame([user]))
    anomaly_predictions.append(pred)

user_features['is_anomaly'] = [p['is_anomaly'] for p in anomaly_predictions]
user_features['anomaly_score'] = [p['anomaly_score'] for p in anomaly_predictions]

# Anomalias detectadas
anomalies = user_features[user_features['is_anomaly'] == True]

print(f"\n🔍 ANOMALIAS DETECTADAS: {len(anomalies)} colaboradores")
print("\nColaboradores com comportamento atípico:")
print(anomalies[['id', 'totalXp', 'nivel', 'nivelEstresse_mean', 'anomaly_score']].head())

# Visualização
fig = px.scatter_3d(
    user_features,
    x='totalXp',
    y='nivelFoco_mean',
    z='nivelEstresse_mean',
    color='is_anomaly',
    title='Detecção de Anomalias (3D)',
    labels={'totalXp': 'XP', 'nivelFoco_mean': 'Foco', 'nivelEstresse_mean': 'Estresse'},
    color_discrete_map={True: 'red', False: 'blue'}
)
fig.show()

print("\n⚠️  INSIGHT PARA GESTOR:")
print("   → Investigar mudanças bruscas de comportamento")
print("   → Possíveis causas: Sobrecarga, problemas pessoais, desmotivação")
print("   → Ação: Conversa individual para entender contexto")

## 10. Dashboard Executivo para Gestores 📊

Visão consolidada por equipe.

In [ ]:
# Métricas por equipe
team_metrics = user_features.groupby('idEquipe').agg({
    'id': 'count',
    'totalXp': 'mean',
    'nivelFoco_mean': 'mean',
    'nivelEstresse_mean': 'mean',
    'burnout_risk_score': 'mean',
    'is_anomaly': 'sum'
}).rename(columns={
    'id': 'total_colaboradores',
    'totalXp': 'xp_medio',
    'burnout_risk_score': 'risco_medio_burnout',
    'is_anomaly': 'anomalias_detectadas'
})

# Merge com nomes
team_metrics = team_metrics.join(equipes.set_index('id')['nome'])

print("\n📊 DASHBOARD EXECUTIVO POR EQUIPE:")
print(team_metrics)

# Gráfico comparativo
fig = go.Figure()
fig.add_trace(go.Bar(
    x=team_metrics['nome'],
    y=team_metrics['risco_medio_burnout'],
    name='Risco Médio de Burnout',
    marker_color='indianred'
))

fig.update_layout(
    title='Risco de Burnout por Equipe',
    xaxis_title='Equipe',
    yaxis_title='Risk Score Médio',
    height=500
)
fig.show()

# Equipe com maior risco
worst_team = team_metrics.loc[team_metrics['risco_medio_burnout'].idxmax()]
print(f"\n🚨 ATENÇÃO GESTOR:")
print(f"   → Equipe '{worst_team['nome']}' apresenta maior risco médio de burnout")
print(f"   → Risk Score: {worst_team['risco_medio_burnout']:.1f}/100")
print(f"   → {worst_team['anomalias_detectadas']} anomalias detectadas")
print(f"   → Ação prioritária: Revisar carga de trabalho e oferecer suporte")

## 11. Resumo de Insights Principais

### Para Colaboradores:

1. **Risco de Burnout**: Monitore seu score e siga as recomendações personalizadas
2. **Melhor Horário**: Estude quando seu foco está no pico (dados individualizados)
3. **Previsão de Performance**: Acompanhe sua evolução e metas
4. **Recomendações de Cursos**: Trilha personalizada baseada em seu perfil
5. **Bem-Estar Impacta**: Dormir bem aumenta sua performance em até 20%

### Para Gestores:

1. **Identificação de Riscos**: Sistema automatizado de alerta para burnout e abandono
2. **Perfis de Equipe**: Diferentes personas requerem estratégias diferentes
3. **Intervenções Proativas**: Agir ANTES do problema se agravar
4. **ROI de Bem-Estar**: Dados provam que investir em saúde mental aumenta performance
5. **Comparação de Equipes**: Benchmarking interno para identificar best practices

---

## Próximos Passos:

- ✅ Integrar modelos ML na API FastAPI (endpoints prontos)
- ✅ Dashboard interativo no frontend
- ✅ Sistema de notificações automáticas
- ✅ Retreinamento periódico dos modelos
- ✅ A/B testing de intervenções

---

**Desenvolvido para FIAP Global Solution 2025.2**

*"O futuro do trabalho será tão humano quanto as ideias que o constroem."*